# Introduction to Panda 3D Game Engine

This notebook provides an introduction to the Panda3D game engine. We will build a simple demo application that gives a gentle introduction to the basics.

We will create a simple program containing some grass and a panda moving around

## ShowBase

The ShowBase class loads most of the other Panda3D modules and causes the 3D window to appear. This code provides a simple example where there is nothing to render, so a blank window will be shown.

In [1]:
from direct.showbase.ShowBase import ShowBase
 
class MyApp(ShowBase):
 
    def __init__(self):
        ShowBase.__init__(self)
 
app = MyApp()
#app.run()
for i in range(10):
    taskMgr.step()

The `app.run()` call in ShowBase contains the Panda3D main loop, which renders a frame, handles the background tasks and then repeats. It doesn't return, and so in most cases it can be called only once at the end of the script. Inside it essentially contains an infinite loop calling `taskMgr.step()`, so here we have manually called this to prevent the infinite loop. The application can then be destroyed using the `app.destroy` function.

In [2]:
app.destroy()

## The Scene Graph

Panda3D contains a data structure called the *Scene Graph*. The scene graph is a tree containing all models that need to be rendered.

To install the grassy scenery model into the Scene Graph, we use the method `reparentTo()` to put the model into the scene graph. The root of the tree is `render`, so the call is `reparentTo(self.render)`

In [3]:
class MyApp(ShowBase):
    
    def __init__(self):
        ShowBase.__init__(self)
        
        # Load the environment model.
        self.scene = self.loader.loadModel("models/environment")
        # Reparent the model to render.
        self.scene.reparentTo(self.render)
        # Apply scale and position transforms on the model.
        self.scene.setScale(0.25, 0.25, 0.25)
        self.scene.setPos(-8, 42, 0)
        
app = MyApp()
#app.run()
for i in range(10):
    taskMgr.step()

The above code will render the environment model. The `setScale()` and `setPos()` methods reposition and rescale the environment to fit in our window. Notice that the objects appear to be floating as the camera is set slightly below the ground, and back-face culling is making the ground invisible to us. By repositioning the camera, the terrain will look better.

In [4]:
app.destroy()

## Moving the Camera

By default Panda3D runs a task that allows you to move the camera using the mouse.

| Key            | Action                     |
|--------------  |----------------------------|
| Left Button    | Pan Left and Right         |
| Right Button   | Move forwards and backwards|
| Middle Button  | Rotate around the origin   |

This can be trialled by running the previous code with `app.run()` instead of `taskMgr.step()`. Note that running this will require the kernel to be restarted in the notebook.

This control method is useful, but can be awkward. Instead, we can write a *task* that controls the camera, updating its position every frame.

In [5]:
from math import pi, sin, cos
 
from direct.showbase.ShowBase import ShowBase
from direct.task import Task
 
class MyApp(ShowBase):
    def __init__(self):
        ShowBase.__init__(self)
 
        # Load the environment model.
        self.scene = self.loader.loadModel("models/environment")
        # Reparent the model to render.
        self.scene.reparentTo(self.render)
        # Apply scale and position transforms on the model.
        self.scene.setScale(0.25, 0.25, 0.25)
        self.scene.setPos(-8, 42, 0)
 
        # Add the spinCameraTask procedure to the task manager.
        self.taskMgr.add(self.spinCameraTask, "SpinCameraTask")
 
    # Define a procedure to move the camera.
    def spinCameraTask(self, task):
        angleDegrees = task.time * 6.0
        angleRadians = angleDegrees * (pi / 180.0)
        self.camera.setPos(20 * sin(angleRadians), -20.0 * cos(angleRadians), 3)
        self.camera.setHpr(angleDegrees, 0, 0)
        return Task.cont
 
app = MyApp()
#app.run()
for i in range(1000):
    taskMgr.step()
app.destroy()

By adding the task to the taskMgr, the function gets called every time `taskMgr.step()` is called. By returning `Task.cont` the task manager will continue to call the function every frame. The `spinCameraTask` spins the camera by calculating the position based on the time the program has been running (`task.time`). 

## Actors

The `Actor` class is for animated models. We can use `loadModel()` for loading static models such as environments, but use the `Actor` class when they need to move.

An actor consists of a file containing the model and a Python dictionary containing the names of the files containing the animations. 

In [6]:
from math import pi, sin, cos
 
from direct.showbase.ShowBase import ShowBase
from direct.task import Task
from direct.actor.Actor import Actor
 
class MyApp(ShowBase):
    def __init__(self):
        ShowBase.__init__(self)
 
        # Load the environment model.
        self.scene = self.loader.loadModel("models/environment")
        # Reparent the model to render.
        self.scene.reparentTo(self.render)
        # Apply scale and position transforms on the model.
        self.scene.setScale(0.25, 0.25, 0.25)
        self.scene.setPos(-8, 42, 0)
 
        # Add the spinCameraTask procedure to the task manager.
        self.taskMgr.add(self.spinCameraTask, "SpinCameraTask")
 
        # Load and transform the panda actor.
        self.pandaActor = Actor("models/panda-model",
                                {"walk": "models/panda-walk4"})
        self.pandaActor.setScale(0.005, 0.005, 0.005)
        self.pandaActor.reparentTo(self.render)
        # Loop its animation.
        self.pandaActor.loop("walk")
 
    # Define a procedure to move the camera.
    def spinCameraTask(self, task):
        angleDegrees = task.time * 6.0
        angleRadians = angleDegrees * (pi / 180.0)
        self.camera.setPos(20 * sin(angleRadians), -20.0 * cos(angleRadians), 3)
        self.camera.setHpr(angleDegrees, 0, 0)
        return Task.cont

app = MyApp()
#app.run()
for i in range(1000):
    taskMgr.step()
app.destroy()

## Intervals

Intervals describe a process of changing a property over time from one value to another. Starting an interval starts a background process that modifies the property over the specified period of time.

Several intervals can be executed in order by putting them into a `Sequence`.

The code can be adjusted to make the panda move across the scene back and forth.

In [7]:
from math import pi, sin, cos
 
from direct.showbase.ShowBase import ShowBase
from direct.task import Task
from direct.actor.Actor import Actor
from direct.interval.IntervalGlobal import Sequence
from panda3d.core import Point3
 
class MyApp(ShowBase):
    def __init__(self):
        ShowBase.__init__(self)
 
        # Disable the camera trackball controls.
        self.disableMouse()
 
        # Load the environment model.
        self.scene = self.loader.loadModel("models/environment")
        # Reparent the model to render.
        self.scene.reparentTo(self.render)
        # Apply scale and position transforms on the model.
        self.scene.setScale(0.25, 0.25, 0.25)
        self.scene.setPos(-8, 42, 0)
 
        # Add the spinCameraTask procedure to the task manager.
        self.taskMgr.add(self.spinCameraTask, "SpinCameraTask")
 
        # Load and transform the panda actor.
        self.pandaActor = Actor("models/panda-model",
                                {"walk": "models/panda-walk4"})
        self.pandaActor.setScale(0.005, 0.005, 0.005)
        self.pandaActor.reparentTo(self.render)
        # Loop its animation.
        self.pandaActor.loop("walk")
 
        # Create the four lerp intervals needed for the panda to
        # walk back and forth.
        pandaPosInterval1 = self.pandaActor.posInterval(13,
                                                        Point3(0, -10, 0),
                                                        startPos=Point3(0, 10, 0))
        pandaPosInterval2 = self.pandaActor.posInterval(13,
                                                        Point3(0, 10, 0),
                                                        startPos=Point3(0, -10, 0))
        pandaHprInterval1 = self.pandaActor.hprInterval(3,
                                                        Point3(180, 0, 0),
                                                        startHpr=Point3(0, 0, 0))
        pandaHprInterval2 = self.pandaActor.hprInterval(3,
                                                        Point3(0, 0, 0),
                                                        startHpr=Point3(180, 0, 0))
 
        # Create and play the sequence that coordinates the intervals.
        self.pandaPace = Sequence(pandaPosInterval1,
                                  pandaHprInterval1,
                                  pandaPosInterval2,
                                  pandaHprInterval2,
                                  name="pandaPace")
        self.pandaPace.loop()
 
    # Define a procedure to move the camera.
    def spinCameraTask(self, task):
        angleDegrees = task.time * 6.0
        angleRadians = angleDegrees * (pi / 180.0)
        self.camera.setPos(20 * sin(angleRadians), -20.0 * cos(angleRadians), 3)
        self.camera.setHpr(angleDegrees, 0, 0)
        return Task.cont
 
app = MyApp()
#app.run()
for i in range(1500):
    taskMgr.step()
app.destroy()

When `pandaPosInterval1` is started it will gradually adjust the position of the panda from (0,10,0) to (0,-10,0) over a period of 13 seconds. The other intervals work in a similar method, and when put together into a sequence it creates the animation of the panda walking back and forth